# **(ETL Rest Wellness and Lifestyle project)**

## Objectives

* Extract: Loading the CSV file into a Pandas DataFrame 
* Transform: Handle missing values, convert data types, normalize column names, remove duplicates
* Load: Save the cleaned dataset to a new CSV file

## Inputs

* Sleep Health and Lifestyle Dataset from: https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset/data  

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* Sleep Health and Lifestyle Dataset is synthetic and was created by Laksika Tharmalingam for illustrative purposes.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\Ewa\\Documents\\vscode-projects\\Rest_Wellness_and_Lifestyle\\02_jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("Rest_Wellness_and_Lifestyle")

Rest_Wellness_and_Lifestyle


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\Ewa\\Documents\\vscode-projects\\Rest_Wellness_and_Lifestyle'

# Section 1

Import python's libraries

In [33]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"


---

# # Step 1: Extract - Loading and Initial Exploration

In [5]:
# Load the data
df = pd.read_csv("/Users/Ewa/Documents/vscode-projects/Rest_Wellness_and_Lifestyle/01_data/Sleep_health_and_lifestyle_raw_dataset.csv")
# Create a copy of the data to avoid modifying the original dataset
df_copy = df.copy()

In [6]:
# Display basic information about the dataset
df_copy.info()
df_copy.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           155 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [8]:
# Get categorical statistics
categorical_stats = df_copy.describe(include='object')
display(categorical_stats)

,Gender,Occupation,BMI Category,Blood Pressure,Sleep Disorder
count,374,374,374,374,155
unique,2,11,4,25,2
top,Male,Nurse,Normal,130/85,Sleep Apnea
freq,189,73,195,99,78


Observations: 

* Gender:

Males are slightly more represented in the dataset.

* Occupation:

11 occupations in the dataset. "Nurse" is the most frequent occupation (potential overrepresentation problem)

* BMI Category:

Four categories: Likely "Underweight," "Normal," "Overweight," and "Obese."
"Normal" is the most frequent category, indicating a relatively healthy BMI distribution in the sample.

* Blood Pressure:

25 unique blood pressure readings. "130/85" is the most frequent reading

* Sleep Disorder:

Three categories: "None," "Insomnia," and "Sleep Apnea." "Sleep Apnea" is the most commonly reported sleep disorder among individuals with recorded disorders.

In [9]:
# Check for missing values in the dataset (code written with Copilot help)
missing_values = df_copy.isnull().sum() / len(df_copy) * 100
missing_values = missing_values[missing_values > 0] # Keep only columns with missing values > 0
missing_values = missing_values.sort_values(ascending=False)    # Sort in descending order
missing_values = missing_values.to_frame()  # Convert to DataFrame  
display(missing_values)

,0
Sleep Disorder,58.55615


In [10]:
# Replacing missing values in the sleep disorder column with 'None'. I will assume that missing values in this column mean that the respondent does not have any sleep disorders. 
df_copy['Sleep Disorder'] = df_copy['Sleep Disorder'].fillna('None')    # Replace missing values with 'None'  

#Check if the missing values have been replaced
missing_values = df_copy.isnull().sum() / len(df_copy) * 100    # Calculate missing values percentage
missing_values = missing_values[missing_values > 0]    # Keep only columns with missing values > 0
missing_values = missing_values.sort_values(ascending=False)    # Sort in descending order
missing_values = missing_values.to_frame()    # Convert to DataFrame
display(missing_values)


,0


In [11]:
#Check for duplicate rows in the dataset
duplicate_rows = df_copy.duplicated().sum() # Count of duplicate rows
display(duplicate_rows) # Display the count of duplicate rows   


0

In [15]:
# Identify categoricaL columns in the dataset
categorical_columns = df_copy.select_dtypes(include=['object']).columns
categorical_columns

Index(['Gender', 'Occupation', 'BMI Category', 'Blood Pressure',
       'Sleep Disorder'],
      dtype='object')

In [16]:
# Display unique values in each categorical column
for column in categorical_columns:
    unique_values = df_copy[column].unique()
    print(f"Unique values in {column}: {unique_values}")

Unique values in Gender: ['Male' 'Female']
Unique values in Occupation: ['Software Engineer' 'Doctor' 'Sales Representative' 'Teacher' 'Nurse'
 'Engineer' 'Accountant' 'Scientist' 'Lawyer' 'Salesperson' 'Manager']
Unique values in BMI Category: ['Overweight' 'Normal' 'Obese' 'Normal Weight']
Unique values in Blood Pressure: ['126/83' '125/80' '140/90' '120/80' '132/87' '130/86' '117/76' '118/76'
 '128/85' '131/86' '128/84' '115/75' '135/88' '129/84' '130/85' '115/78'
 '119/77' '121/79' '125/82' '135/90' '122/80' '142/92' '140/95' '139/91'
 '118/75']
Unique values in Sleep Disorder: ['None' 'Sleep Apnea' 'Insomnia']


In [17]:
# BMI Category has Normal and Normal Weight which are identical (It requires merging)
df_copy['BMI Category'] = df_copy['BMI Category'].replace({'Normal Weight': 'Normal'})
df_copy['BMI Category'].value_counts().reset_index()

,BMI Category,count
0,Normal,216
1,Overweight,148
2,Obese,10


NOTE

In [18]:
# Check for unique values in 'Occupation' column
df_copy['Occupation'].value_counts().reset_index()  # Display unique values in 'Occupation' column


,Occupation,count
0,Nurse,73
1,Doctor,71
2,Engineer,63
3,Lawyer,47
4,Teacher,40
5,Accountant,37
6,Salesperson,32
7,Software Engineer,4
8,Scientist,4
9,Sales Representative,2


Examination:

"Engineeer" could be merged with "Software Engineer"

"Sales Representative" could be merged with "Salesperson"

The "Manager" and "Scientist" professions are underrepresented in the dataset which would make meaningful analysis difficult. Therfore, I decided to remove the corresponding rows from the DataFrame.

In [19]:
df_copy["Occupation"] = df_copy["Occupation"].replace({'Sales Representative': 'Salesperson'}) 
df_copy["Occupation"] = df_copy["Occupation"].replace({'Software Engineer': 'Engineer'})

index_drop = df_copy[df_copy['Occupation'].isin(['Manager', 'Scientist'])].index
df_copy = df_copy.drop(index_drop)

df_copy['Occupation'].value_counts()

Occupation
Nurse          73
Doctor         71
Engineer       67
Lawyer         47
Teacher        40
Accountant     37
Salesperson    34
Name: count, dtype: int64

Blood Pressure Measurement

Blood pressure is measured using two numbers:

Systolic blood pressure (first number) – Represents the pressure exerted by blood against artery walls when the heart contracts.

Diastolic blood pressure (second number) – Represents the pressure exerted by blood against artery walls when the heart is at rest between beats.

The dataset contains a single column for both systolic and diastolic blood pressure. This means that to categorize blood pressure as normal, elevated, or high, I would need to split the data in the 'Blood Pressure' column.

In [23]:
# Split the 'Blood Pressure' column into 'Systolic Pressure' and 'Diastolic Pressure' columns

df_copy[['Systolic Pressure', 'Diastolic Pressure']] = df_copy['Blood Pressure'].str.split('/', expand=True)
df_copy['Systolic Pressure'] = pd.to_numeric(df_copy['Systolic Pressure'])
df_copy['Diastolic Pressure'] = pd.to_numeric(df_copy['Diastolic Pressure'])

# Creating the Blood Pressure Category column
blood_pressure_conditions = [
    (df_copy['Systolic Pressure'] < 120) & (df_copy['Diastolic Pressure'] < 80),
    (df_copy['Systolic Pressure'].between(120, 140)) & (df_copy['Diastolic Pressure'] < 90),
    (df_copy['Systolic Pressure'] >= 140) & (df_copy['Diastolic Pressure'] >= 90) | (df_copy['Diastolic Pressure'] >= 80)
]

labels = ['Optimal', 'Normal', 'Hypertension']

df_copy['Blood Pressure Category'] = np.select(blood_pressure_conditions, labels, default='Undefined')

df_copy.head(5)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder,Systolic Pressure,Diastolic Pressure,Blood Pressure Category
0,1,Male,27,Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None,126,83,Normal
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None,125,80,Normal
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None,125,80,Normal
3,4,Male,28,Salesperson,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea,140,90,Hypertension
4,5,Male,28,Salesperson,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea,140,90,Hypertension


* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

## EDA 

In [24]:
# Describe basic statistics of the dataset

summary_stats = df_copy.describe()
styled_summary = summary_stats.style.background_gradient(cmap='Blues')
display(styled_summary)

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systolic Pressure,Diastolic Pressure
count,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000,369.000000
mean,188.506775,42.271003,7.144986,7.338753,59.379404,5.368564,70.062331,6836.314363,128.552846,84.647696
std,108.132495,8.682957,0.791829,1.175677,20.862834,1.775341,4.054508,1620.028673,7.796184,6.200938
min,1.000000,27.000000,5.900000,4.000000,30.000000,3.000000,65.000000,3000.000000,115.000000,75.000000
25%,97.000000,36.000000,6.500000,6.000000,45.000000,4.000000,68.000000,6000.000000,125.000000,80.000000
50%,189.000000,43.000000,7.200000,7.000000,60.000000,5.000000,70.000000,7000.000000,130.000000,85.000000
75%,282.000000,50.000000,7.800000,8.000000,75.000000,7.000000,72.000000,8000.000000,135.000000,90.000000
max,374.000000,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000,142.000000,95.000000


Dataset Summary:

* Age:

Mean (42.27) / Median (43): The dataset reflects a somewhat older adult population . The close values of the mean and median suggest the age distribution is fairly symmetrical.
Standard Deviation (8.68): The age distribution has a moderate spread, with a mix of younger and older individuals around the average.
Range (27 - 59): The dataset captures a wide age range of adults.

* Sleep Duration:

Mean (7.14) / Median (7.20): On average, individuals in the dataset get about 7 hours of sleep per day. The proximity of the mean and median suggests a relatively symmetrical distribution.
Range (5.9 - 8.5): There’s some variation in sleep duration, but most individuals sleep within a fairly narrow range.

* Quality of Sleep:

Mean (7.33) / Median (7): The average sleep quality rating is above the midpoint of the 1-10 scale. The slightly higher mean compared to the median suggests the distribution may be slightly right-skewed (positive skew), meaning that while most individuals report average to good sleep quality, a few individuals with significantly higher sleep quality ratings are pulling the mean up.

* Physical Activity Level:

Mean (59.37) / Median (60): On average, individuals engage in around an hour of physical activity per day.
Range (30 - 90): There is considerable variation in activity levels, with some individuals being significantly more active than others.

* Stress Level:

Mean (5.36) / Median (5): The average stress level is slightly above the midpoint of the 1-10 scale. The lower mean compared to the median suggests a possible skew towards higher stress levels for certain individuals.
 
 * Heart Rate:

Mean (70.06) / Median (70): The average resting heart rate is approximately 70 beats per minute, within the normal range for adults. The near-identical mean and median suggest a balanced distribution.

* Daily Steps:

Mean (6836.31) / Median (7000): On average, individuals take nearly 7,000 steps per day.
Range (3000 - 10000): There’s considerable variation in daily step counts, indicating varying levels of activity among individuals.

In [25]:
# Check for the outliers in the dataset using Z-scores

# Select only numerical columns to avoid issues with non-numeric data
numeric_df = df_copy.select_dtypes(include=[np.number])

# Compute Z-scores for numerical columns
z_scores = np.abs((numeric_df - numeric_df.mean()) / numeric_df.std())

# Identify outliers (Z-score > 3)
outliers = z_scores > 3

# Count the total number of outliers in the dataset
num_outliers = outliers.sum().sum()

# Display the number of outliers
num_outliers


9

In [34]:
# Plotting outliers on a scatter plot
# Select only numerical columns to avoid issues with non-numeric data
numeric_df = df_copy.select_dtypes(include=[np.number])

# Compute Z-scores for numerical columns
z_scores = np.abs((numeric_df - numeric_df.mean()) / numeric_df.std())

# Identify outliers (Z-score > 3)
outliers = z_scores > 3

# Marking outliers
outlier_indices = np.where(outliers)  # Extract indices of outliers

# Create a DataFrame to separate outliers and normal points
outliers_df = numeric_df.iloc[outlier_indices[0]]
normal_df = numeric_df.drop(outlier_indices[0])

# Plotting the normal data points and outliers on a scatter plot
fig = px.scatter(x=numeric_df.iloc[:, 0], y=numeric_df.iloc[:, 1], title="Scatter Plot with Outliers", 
                 labels={'x': numeric_df.columns[0], 'y': numeric_df.columns[1]})

# Add the outliers as red points
fig.add_scatter(x=outliers_df.iloc[:, 0], y=outliers_df.iloc[:, 1], mode='markers', name="Outliers", 
                marker=dict(color='red', size=10))

# Show the plot
fig.show()

---

In [38]:
# Plotting outliers on a heatmap
# Create a heatmap of Z-scores  
fig = px.imshow(z_scores, labels={'color': 'Z-Score'}, title="Heatmap of Z-Scores")
fig.show()



# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
